In [1]:
import os
import glob
import librosa
import numpy as np
from tqdm import tqdm # Thêm thư viện này để xem thanh tiến trình (progress bar)
from scipy.signal import butter, lfilter

In [2]:
# --- 0. Định nghĩa các hằng số và thư mục ---
RAW_DATA_DIR = "../data/raw/"
PROCESSED_DATA_DIR = "../data/processed/"
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Ánh xạ nhãn
emotion_map = {
    "01": "neutral", "02": "calm", "03": "happy", "04": "sad",
    "05": "angry", "06": "fearful", "07": "disgust", "08": "surprised"
}

# --- 1. HÀM LỌC (THÊM MỚI) ---
def butter_bandpass_filter(data, lowcut=100.0, highcut=8000.0, fs=22050, order=5):
    """
    Hàm thiết kế và áp dụng bộ lọc bandpass Butterworth.
    """
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y_filtered = lfilter(b, a, data)
    return y_filtered

# --- 2. Hàm trích xuất đặc trưng (ĐÃ CẬP NHẬT) ---
def extract_features_filtered(file_path):
    try:
        # 1. Tải file âm thanh
        y, sr = librosa.load(file_path, sr=22050)
        
        # 2. ÁP DỤNG BỘ LỌC (BƯỚC MỚI)
        y_filtered = butter_bandpass_filter(y, fs=sr)
        
        # 3. Trích xuất đặc trưng từ tín hiệu ĐÃ LỌC (y_filtered)
        mfccs = np.mean(librosa.feature.mfcc(y=y_filtered, sr=sr, n_mfcc=20).T, axis=0)
        rms = np.mean(librosa.feature.rms(y=y_filtered).T, axis=0)
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y_filtered).T, axis=0)
        
        # 4. Kết hợp
        features = np.hstack((mfccs, rms, zcr))
        
        # 5. Lấy nhãn
        filename = os.path.basename(file_path)
        label = emotion_map[filename.split('-')[2]]
        
        return features, label
    
    except Exception as e:
        print(f"Lỗi khi xử lý file {file_path}: {e}")
        return None, None

# --- 3. Vòng lặp xử lý tất cả file ---
all_features = []
all_labels = []

print("Bắt đầu trích xuất đặc trưng (ĐÃ LỌC)...")
file_paths = glob.glob(os.path.join(RAW_DATA_DIR, "Actor_*", "*.wav"))

for file_path in tqdm(file_paths):
    # Gọi hàm mới
    features, label = extract_features_filtered(file_path)
    
    if features is not None:
        all_features.append(features)
        all_labels.append(label)

print(f"\nTrích xuất hoàn tất. Xử lý được {len(all_features)} / {len(file_paths)} file.")

# --- 4. Lưu kết quả ra file (ĐẶT TÊN MỚI) ---
X = np.array(all_features)
y = np.array(all_labels)

print(f"Mảng đặc trưng (X) có hình dạng: {X.shape}")
print(f"Mảng nhãn (y) có hình dạng: {y.shape}")

# Lưu vào thư mục processed với tên mới
np.save(os.path.join(PROCESSED_DATA_DIR, 'features_filtered.npy'), X) # <-- TÊN MỚI
np.save(os.path.join(PROCESSED_DATA_DIR, 'labels_filtered.npy'), y) # <-- TÊN MỚI

print("Đã lưu features_filtered.npy và labels_filtered.npy vào 'data/processed/'")

Bắt đầu trích xuất đặc trưng (ĐÃ LỌC)...


100%|██████████| 1440/1440 [00:24<00:00, 58.05it/s]


Trích xuất hoàn tất. Xử lý được 1440 / 1440 file.
Mảng đặc trưng (X) có hình dạng: (1440, 22)
Mảng nhãn (y) có hình dạng: (1440,)
Đã lưu features_filtered.npy và labels_filtered.npy vào 'data/processed/'
